**Mount the drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Unzip the data**

In [0]:
zip_path = '/content/gdrive/My Drive/Dataset.zip'
!unzip '$zip_path' 
!unzip -q '/content/SIMS dataset/images.zip' 
!unzip -q '/content/SIMS dataset/Annotations_in_3_formats.zip' 

Archive:  /content/gdrive/My Drive/Dataset.zip
  inflating: SIMS dataset/test.txt   
  inflating: SIMS dataset/validation.txt  
  inflating: SIMS dataset/training.txt  
  inflating: SIMS dataset/Assignment-2_modified scope.pdf  
  inflating: SIMS dataset/Annotations_in_3_formats.zip  
  inflating: SIMS dataset/images.zip  


**Copy the Pascal_Voc Annotations to the images folder**

In [0]:
%cp -R "/content/annotations/pascal_voc/Annotations/."  "/content/images"


**Make the directories**

In [0]:
!mkdir Dataset
!mkdir Dataset/train
!mkdir Dataset/train/images
!mkdir Dataset/train/annotations
!mkdir Dataset/valid
!mkdir Dataset/valid/images
!mkdir Dataset/valid/annotations
!mkdir Dataset/test
!mkdir Dataset/test/images
!mkdir Dataset/test/annotations

**Creating CSV files and placing images and labels in the directories**

In [0]:
import shutil
import os
#copy files based on {datasets in txt}

def copy(filepath,dest_dir):
    with open(filepath) as fp:
        for line in fp:
            os.makedirs(dest_dir, exist_ok=True)  # succeeds even if directory exists.
            shutil.copy(line.replace('\n', ''), dest_dir+"/images") #image
            shutil.copy(line.replace('.jpg\n', '.xml'), dest_dir+"/annotations") ##label

#copy files based on {datasets in txt}
copy('/content/SIMS dataset/training.txt','Dataset/train/')
copy('/content/SIMS dataset/validation.txt','Dataset/valid/')
copy('/content/SIMS dataset/test.txt','Dataset/test/')

#from preprocessing.pascal_voc_to_csv import xml_to_csv

import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path,image_path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        #print(xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text
            
            size = root.find('size') #contains height and width of image
            w = int(size.find('width').text)
            h = int(size.find('height').text)
            
            # truncate any bb coordinates that fall outside of the image
            xmin = max(0, xmin)
            ymin = max(0, ymin)
            xmax = min(w, xmax)
            ymax = min(h, ymax)
            
            # # ignore the bounding boxes where the minimum values are larger
            # # than the maximum values and vice-versa due to annotation errors
            if xmin >= xmax or ymin >= ymax:
                continue
            elif xmax <= xmin or ymax <= ymin:
                continue
            
            file_path = image_path+root.find('filename').text+".jpg"
            value = (file_path,
                     xmin,
                     ymin,
                     xmax,
                     ymax,
                     label,
                     )
            xml_list.append(value)
    column_name = ['filename','xmin', 'ymin', 'xmax', 'ymax','class']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    root_path = "Dataset/"
    datasets = ['train','test','valid']
    for ds in datasets:
        #image_path = os.path.join(os.getcwd(), root_path+ds, 'annotations')
        image_path = os.path.join(root_path+ds, 'annotations/')
        print(image_path)
        update_path = os.path.join(root_path+ds, 'images/')
        xml_df = xml_to_csv(image_path,update_path)
        xml_df.to_csv('labels_{}.csv'.format(ds), index=None, header=False)
        print('Successfully converted xml to csv.')

**Clone the Repository**

In [0]:
!git clone https://github.com/arslanamin14/keras-yolo3.git

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


**Run the command for installing the packages**

In [0]:
!pip install -r requirements.txt

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 2.4MB 8.3MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 20.2MB 10.0MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 1.3MB 48.7MB/s 
     |████████████████████████████████| 276kB 54.1MB/s 
     |████████████████████████████████| 3.8MB 52.3MB/s 
     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 512kB 28.1MB/s 
     |████████████████████████████████| 327kB 53.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ebfa8f9ddb60350bd209e4722323aa055a9fa99b44e4c300abe11210999e06d7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp36-none-any.whl size=61682 sha256=8f0a52e2119158acebf42904d2eb750089046b3ee

**Run the command for installing the tensorflow gpu 1.15**

In [0]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 39kB/s 


**Copy the backend.h5 file to the yolov3 folder**

In [0]:
%cp -R "/content/gdrive/My Drive/ACV2dataunzip/backend.h5"  "/content/keras-yolo3" 

**Remove the Config.json file from yoloV3 folder**

In [0]:
!rm -r "/content/keras-yolo3/config.json"

**Copy the Config.json file to yoloV3 folder**

In [0]:
%cp -R "/content/gdrive/My Drive/ACV2/config.json"  "/content/keras-yolo3" 

**Run the gen_anchors.py for generation of anchors**

In [0]:
!cd "/content/keras-yolo3" && python gen_anchors.py -c config.json

not well-formed (invalid token): line 1, column 0
Ignore this bad annotation: /content/Dataset/train/annotations/2286.jpg
/content/Dataset/train/images/0001
/content/Dataset/train/images/0002
/content/Dataset/train/images/0003
/content/Dataset/train/images/0004
/content/Dataset/train/images/0005
/content/Dataset/train/images/0006
/content/Dataset/train/images/0007
/content/Dataset/train/images/0008
/content/Dataset/train/images/0009
/content/Dataset/train/images/0010
/content/Dataset/train/images/0012
/content/Dataset/train/images/0016
/content/Dataset/train/images/0017
/content/Dataset/train/images/0018
/content/Dataset/train/images/0019
/content/Dataset/train/images/0021
/content/Dataset/train/images/0022
/content/Dataset/train/images/0024
/content/Dataset/train/images/0026
/content/Dataset/train/images/0027
/content/Dataset/train/images/0028
/content/Dataset/train/images/0029
/content/Dataset/train/images/0030
/content/Dataset/train/images/0031
/content/Dataset/train/images/0032
/co

**Train the Model**

In [0]:
!cd "/content/keras-yolo3" && python train.py -c config.json

Streaming output truncated to the last 5000 lines.
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
resizing:  128 128
 - 552s - loss: 18.3477 - yolo_layer_1_loss: 0.5791 - yolo_layer_2_loss: 2.6180 - yolo_layer_3_loss: 15.1506

Epo